In [175]:
import pandas as pd

In [176]:
nums = [13, 15, 16, 16, 19, 20, 20, 21, 22, 
        22, 25, 25, 25, 25, 30, 33, 33, 35, 
        35, 35, 35, 36, 40, 45, 46, 52, 70]
df1 = pd.DataFrame()
df1['nums'] = nums

In [177]:
# length of data.
l = df1.size

# cut idx.
q = int(l/3)

# binning & averaging.
df1['3bin_mean'] = 0
df1['3bin_mean'].iloc[0:q] = df1['nums'].iloc[0:q].mean()
df1['3bin_mean'].iloc[q:2*q] = df1['nums'].iloc[q:2*q].mean()
df1['3bin_mean'].iloc[2*q:l] = df1['nums'].iloc[2*q:l].mean()

df1


,nums,3bin_mean
0,13,18.000000
1,15,18.000000
2,16,18.000000
3,16,18.000000
4,19,18.000000
5,20,18.000000
6,20,18.000000
7,21,18.000000
8,22,18.000000
9,22,28.111111


In [178]:
# summary statistics.
df1.describe()

,nums,3bin_mean
count,27.000000,27.000000
mean,29.962963,29.962963
std,12.942124,10.806904
min,13.000000,18.000000
25%,20.500000,18.000000
50%,25.000000,28.111111
75%,35.000000,43.777778
max,70.000000,43.777778


In [179]:
# IQR.
q1 = df1.describe()['nums']['25%']
q2 = df1.describe()['nums']['75%']

# 1.5 IQR.
min_allowed = q1/1.5
max_allowed = q2*1.5

# Mask the dataframe.
outliers = df1['nums'][(df1['nums'] < min_allowed) | (df1['nums'] > max_allowed)]
outliers

0     13
26    70
Name: nums, dtype: int64

In [180]:
# Create min max scaling function.
def min_max_scaler(val):
    r = df1['nums'].max() - df1['nums'].min()
    return (val - df1['nums'].min()) / r

# Scale 35.
min_max_scaler(35)

0.38596491228070173

In [181]:
# Create z score scaling function.
def z_score_scaler(val):
    return (val - df1['nums'].mean()) / df1['nums'].std()

# Scale 35.
z_score_scaler(35)

0.3891970907527787

In [182]:
# Create decimal scaling function.
def decimal_scaler(val):
    return val/100

# Scale 35.
decimal_scaler(35)

0.35

In [183]:
import numpy as np
# Use data from q1 as example.
a = np.array(nums)

# Define variable min max scaler.
def min_max_mapper(np_array, new_min, new_max):
    mn = np_array.min()
    mx = np_array.max()
    r = mx - mn
    a = (np_array - mn) / r
    new_r = new_max - new_min 
    return (a * new_r) + new_min

# Test with data.
min_max_mapper(a, -10, 10)

array([-10.        ,  -9.29824561,  -8.94736842,  -8.94736842,
        -7.89473684,  -7.54385965,  -7.54385965,  -7.19298246,
        -6.84210526,  -6.84210526,  -5.78947368,  -5.78947368,
        -5.78947368,  -5.78947368,  -4.03508772,  -2.98245614,
        -2.98245614,  -2.28070175,  -2.28070175,  -2.28070175,
        -2.28070175,  -1.92982456,  -0.52631579,   1.22807018,
         1.57894737,   3.68421053,  10.        ])

In [184]:
data = {
    'department' : ['sales', 'sales', 'sales', 'systems', 
                    'systems', 'systems', 'systems', 'marketing', 
                    'marketing', 'secretary', 'secretary'],
    'age' : ['31_35', '26_30', '31_35', '21_25', 
             '31_35', '26_30', '41_45', '36_40', 
             '31_35', '46_50', '26_30'],
    'salary' : ['46K_50K', '26K_30K', '31K_35K', '46K_50K', 
                '66K_70K', '46K_50K', '66K_70K', '46K_50K', 
                '41K_45K', '36K_40K', '26K_30K'],
    'status' : ['senior', 'junior', 'junior', 'junior', 
                'senior', 'junior', 'senior', 'senior', 
                'junior', 'senior', 'junior'],
    'count' : [30, 40, 40, 20, 5, 3, 3, 10, 4, 4, 6]
}

df2 = pd.DataFrame(data)
df2

,department,age,salary,status,count
0,sales,31_35,46K_50K,senior,30
1,sales,26_30,26K_30K,junior,40
2,sales,31_35,31K_35K,junior,40
3,systems,21_25,46K_50K,junior,20
4,systems,31_35,66K_70K,senior,5
5,systems,26_30,46K_50K,junior,3
6,systems,41_45,66K_70K,senior,3
7,marketing,36_40,46K_50K,senior,10
8,marketing,31_35,41K_45K,junior,4
9,secretary,46_50,36K_40K,senior,4


In [262]:
def gainer(df, label, counts):
    # Create DataFrame for storing gain.
    gainz = pd.DataFrame(index=['entropy', 'gain'])

    # Find total observations.
    total = df[counts].sum()
    # print(total)
    
    # Find total observations per label.
    sum0 = pd.DataFrame(
        df.groupby(by=label)[counts].apply(
            lambda x: x.sum()
        ))
    # print(sum0)

    # Find label probabilities.
    p_label = sum0 / total
    # print(p_label)

    # Find total entropy.
    total_entrop = float((-p_label * np.log2(p_label)).sum())
    # print(total_entrop)

    # Find attribute columns.
    cols = df.columns.drop(label)
    cols = cols.drop(counts)
    # print(cols)

    for col in cols:
        # Find total observations per bin.
        sum1 = pd.DataFrame(
            df.groupby(by=col)[counts].apply(
                lambda x: x.sum()
            ))

        # Find label totals per bin.
        sum2 = pd.DataFrame(
            df.groupby(by=[col, label])[counts].apply(
                lambda x: x.sum()
            ))

        # Solve for entropy per class
        p_label_class = sum2 / sum1
        H = -p_label_class * np.log2(p_label_class)

        # Solve for expected entropy per class
        entrop = pd.DataFrame(
            H.unstack().apply('sum', axis=1), columns=['H']
            )
        entrop['P[class]'] = sum1 / total
        entrop['E[H]'] = entrop['H'] * entrop['P[class]']
        entropy = entrop['E[H]'].sum()

        gain = total_entrop - entropy
        gainz[col] = [entropy, gain]

    return gainz

g1 = gainer(df2, 'status', 'count')
g1.T

165
        count
status       
junior    113
senior     52
           count
status          
junior  0.684848
senior  0.315152
0.899030771238222
Index(['department', 'age', 'salary'], dtype='object')


,entropy,gain
department,0.850424,0.048607
age,0.474296,0.424735
salary,0.361513,0.537518


In [256]:
g2 = df2.groupby('salary').apply(
    lambda x: gainer(x.drop('salary', axis=1), 'status', 'count')
)

g2

# pd.DataFrame(
#         df2.groupby(by=['salary', 'status']).apply(
#             lambda x: x.sum()
#         ))

department       age
salary                               
26K_30K entropy    0.000000  0.000000
        gain       0.000000  0.000000
31K_35K entropy    0.000000  0.000000
        gain       0.000000  0.000000
36K_40K entropy    0.000000  0.000000
        gain       0.000000  0.000000
41K_45K entropy    0.000000  0.000000
        gain       0.000000  0.000000
46K_50K entropy    0.000000  0.000000
        gain       0.946819  0.946819
66K_70K entropy    0.000000  0.000000
        gain       0.000000  0.000000

In [ ]:
table3 = {
  "gender": ["male","male","female","female",
             "male","female","female","male","female"],
  "age" : ["young","young","young","teenager",
           "young","young","elder","middle age","elder"],
  "income" : ["medium","low","low","medium",
              "high","medium","high","medium","medium"],
  "play golf?" : ["yes","no","no","no",
                  "yes","no","yes","yes","yes"],
  "count" : [30,20,30,20,15,30,13,10,4],
  }
  
df4 = pd.DataFrame(table3)
df4

g1 = gainer(df4, 'gender', 'count')
g1.T